In [ ]:
import os
from collections import OrderedDict
import numpy as np
import cv2
from PIL import Image
from os.path import join

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
matplotlib.rcParams['image.cmap'] = 'gray'
np.set_printoptions(precision=4)
# %matplotlib

In [ ]:
FOLDER =  '../data'
SQUARE_SIZE = 0.159
objpts = [[0,0,0],[1,0,0],[1,1,0],[0,1,0]]
objpts = np.array(objpts)[:,np.newaxis]*SQUARE_SIZE

# Visualize initial pose

In [ ]:
fs_read = cv2.FileStorage(join(FOLDER, "initial_camera_poses.yml"), cv2.FILE_STORAGE_READ)
T = {}
for key in fs_read.root().keys():
    print(key)
    T[key] = fs_read.getNode(key).mat()
ORIGIN = key[:4]

In [ ]:
def drawAxis(T=np.eye(4) ,scale=0.3, colors=['r', 'g', 'b']):
    tvec = T[:3,3]
    R = T[:3,:3]
    start = tvec.flatten()[np.newaxis].repeat(3, axis=0)
    end = start + scale * R.T
    for s, e, c in zip(start, end, colors):
        ax.plot([s[0], e[0]], [s[1], e[1]], [s[2], e[2]], c=c)
        
%matplotlib notebook

In [ ]:
# 3D Plot
fig = plt.figure(figsize=(8, 6))
ax = Axes3D(fig)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
#ax.axis('off')
c = ['c', 'y', 'b', 'b']

# Camera
for i in range(4):
    colors = ['r', 'g', c[i]]
    key = f'{ORIGIN}img{i}'
    drawAxis(T[key], 0.3, colors)
    
# tag
for i in range(4):
    if int(ORIGIN[-1]) == i:
        continue
    key = f'{ORIGIN}tag{i}'
    drawAxis(T[key], 0.1)
    pts = T[key][:3,:3].dot(objpts.squeeze().T) + T[key][:3,3:4]
    ax.scatter(pts[0], pts[1], zs=pts[2])

# Origin
drawAxis(scale=0.15)
#pts origin
ax.scatter(objpts[:,:,0].flatten(), objpts[:,:,1].flatten(), zs=objpts[:,:,2].flatten())

# adjust scale
start = np.zeros((3, 3)) - 1.0
end = start + 2.0 * np.eye(3)
for s, e in zip(start, end):
    ax.plot([s[0], e[0]], [s[1], e[1]], [s[2], e[2]], c='none')
    




# Visualize refined poses

In [ ]:
fs_read = cv2.FileStorage(join(FOLDER, "refined_camera_poses.yml"), cv2.FILE_STORAGE_READ)
T = {}
for key in fs_read.root().keys():
    print(key)
    T[key] = fs_read.getNode(key).mat()

In [ ]:
def getRotation(angle):
    rot = np.array([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle), np.cos(angle), 0],
        [0, 0, 1]]
    )
    return rot

In [ ]:
fs_read = cv2.FileStorage(join(FOLDER, "refined_params.yml"), cv2.FILE_STORAGE_READ)
tag_pts = {}
for key in fs_read.root().keys():
    node = fs_read.getNode(key)
    print(key)
    param = [node.at(i).real() for i in range(node.size())]
    tag_pt  = np.array([param[0], param[1], 0])[:, np.newaxis]  + getRotation(param[2]).dot(objpts.T.squeeze())
    tag_pts[key] = tag_pt

In [ ]:
# 3D Plot
fig = plt.figure(figsize=(8, 6))
ax = Axes3D(fig)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
#ax.axis('off')
c = ['c', 'y', 'b', 'b']

# Camera
for i in range(4):
    colors = ['r', 'g', c[i]]
    key = f'originimg{i}'
    drawAxis(T[key], 0.3, colors)
    
# tag
for i in range(0, 4):
    key = f'tag{i}'
    pts = tag_pts[key]
    ax.scatter(pts[0], pts[1], zs=pts[2])

# Origin
drawAxis(scale=0.15)

# adjust scale
start = np.zeros((3, 3)) - 1.0
end = start + 2.0 * np.eye(3)
for s, e in zip(start, end):
    ax.plot([s[0], e[0]], [s[1], e[1]], [s[2], e[2]], c='none')
    




# Visualize final poses

In [ ]:
fs_read = cv2.FileStorage(join(FOLDER, "final_camera_poses.yml"), cv2.FILE_STORAGE_READ)
T = {}
for key in fs_read.root().keys():
    print(key)
    T[key] = fs_read.getNode(key).mat()

In [ ]:
# 3D Plot
fig = plt.figure(figsize=(8, 6))
ax = Axes3D(fig)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
#ax.axis('off')
# colors = ['r', 'g', 'b']
c = ['c', 'y', 'b', 'b']

# Camera
for i in range(4):
    colors = ['r', 'g', c[i]]
    key = f'originimg{i}'
    drawAxis(T[key], 0.3, colors)

# Origin
drawAxis(scale=0.15)

# adjust scale
start = np.zeros((3, 3)) - 1.0
end = start + 2.0 * np.eye(3)
for s, e in zip(start, end):
    ax.plot([s[0], e[0]], [s[1], e[1]], [s[2], e[2]], c='none')
    


